In [4]:
import pandas as pd
import sklearn.model_selection 
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [21]:
## load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
### preprocessin data and dropping irrelevant columns
data = data.drop([['RowNumber','CustomerId','Surname']],axis=1)

KeyError: "[('RowNumber', 'CustomerId', 'Surname')] not found in axis"

In [27]:
### encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [36]:
##OHe "geography"
from sklearn.preprocessing import OneHotEncoder
a = OneHotEncoder()
b = a.fit_transform(data[["Geography"]])
b

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [40]:
a.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [44]:
c=pd.DataFrame(b.toarray(),columns=a.get_feature_names_out(["Geography"]))
c

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [46]:
data=pd.concat([data.drop("Geography",axis=1),c],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [48]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('b.pkl','wb') as file:
    pickle.dump(b,file)

In [51]:
##Divide dataset into independent and dependent features
X = data.drop("Exited", axis=1)
y = data["Exited"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

##Scaling Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.21835119,  0.91186722,  1.91661905, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [ 2.05728037,  0.91186722,  0.20210899, ..., -0.99946681,
         1.73082   , -0.57735027],
       [ 0.75860157, -1.09665089, -0.75039661, ...,  1.00053348,
        -0.57776083, -0.57735027],
       ...,
       [ 0.86249588, -1.09665089, -0.08364269, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [ 0.15601461,  0.91186722,  0.3926101 , ...,  1.00053348,
        -0.57776083, -0.57735027],
       [ 0.46769752,  0.91186722,  1.15461458, ..., -0.99946681,
         1.73082   , -0.57735027]])

In [52]:
with open("scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

In [ ]:
def generate_sequence(n):
    return ''.join(str(i) for i in range(1, n+1))

print(generate_sequence(5))

In [67]:
def generate_sequence(n):
    return ''.join(str(i) for i in range(1, n+1))

print(generate_sequence(5))

12345


In [1]:
####  ANN implementstion
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
### build our ann model
model=eq